In [ ]:
from iobjectspy import (datasetvector_to_numpy_array,
                        numpy_array_to_datasetvector,
                        datasetraster_to_numpy_array,
                        numpy_array_to_datasetraster)
import numpy as np
import os

# 设置示例数据路径
example_data_dir = ''

# 设置结果输出路径
out_dir = os.path.join(example_data_dir, 'out')

if not os.path.exists(out_dir):
    os.makedirs(out_dir)


def vector_to_numpy_test():
    """读取矢量数据集 Town_P 到 numpy 数组中"""

    narray1 = datasetvector_to_numpy_array(os.path.join(example_data_dir, 'example_data.udb/Landuse_R'), export_spatial=True)
    print(narray1.dtype)
    print(narray1[:10])

    narray = datasetvector_to_numpy_array(os.path.join(example_data_dir, 'example_data.udb/Town_P'), export_spatial=True)

    if narray is None:
        print('读取矢量数据集到 numpy 数组失败')
    else:
        print('ndarray.ndim : ' + str(narray.ndim))
        print('ndarray.dtype : ' + str(narray.dtype))
        print(narray[:10])
        print(narray['SmX'][:10])
        print('读取矢量数据集到 numpy 数组成功')

        xy_array = np.c_[narray['SmX'], narray['SmY']][:10]
        print(xy_array.ndim)
        print(xy_array.dtype)
        print(xy_array)

        try:
            # 使用 hdbscan 做层次聚类,并将结果在 matplotlib 显示

            % matplotlib inline
            from hdbscan import HDBSCAN
            import matplotlib.pyplot as plt

            xy_c = np.c_[narray['SmX'], narray['SmY']]
            hdb = HDBSCAN(min_cluster_size=10).fit(xy_c)
            hdb_labels = hdb.labels_
            n_clusters_hdb_ = len(set(hdb_labels)) - (1 if -1 in hdb_labels else 0)

            hdb_unique_labels = set(hdb_labels)
            hdb_colors = plt.cm.Spectral(np.linspace(0, 1, len(hdb_unique_labels)))
            fig = plt.figure(figsize=plt.figaspect(1))
            hdb_axis = fig.add_subplot('111')
            for k, col in zip(hdb_unique_labels, hdb_colors):
                if k == -1:
                    col = 'k'
                hdb_axis.plot(xy_c[hdb_labels == k, 0], xy_c[hdb_labels == k, 1], 'o', markerfacecolor=col,
                              markeredgecolor='k', markersize=6)

            hdb_axis.set_title('HDBSCAN\nEstimated number of clusters: %d' % n_clusters_hdb_)
            plt.show()

        except ImportError:
            pass


def numpy_to_vector_test():
    """写入数据到矢量数据集中"""

    narray = np.empty(10, dtype=[('ID', 'int32'), ('X', 'float64'), ('Y', 'float64'), ('NAME', 'U10')])

    narray[0] = 1, 116.380351, 39.93393099, '什刹海'
    narray[1] = 2, 116.365305, 39.89622499, '广安门内'
    narray[2] = 3, 116.427342, 39.89467499, '崇文门外'
    narray[3] = 4, 116.490881, 39.96567299, '酒仙桥'
    narray[4] = 5, 116.447486, 39.93767799, '三里屯'
    narray[5] = 6, 116.347435, 40.08078599, '回龙观'
    narray[6] = 7, 116.407196, 39.83895899, '大红门'
    narray[7] = 8, 116.396915, 39.88371499, '天桥'
    narray[8] = 9, 116.334522, 40.03594199, '清河'
    narray[9] = 10, 116.03008, 39.87852799, '潭柘寺'

    print(narray)
    result = numpy_array_to_datasetvector(narray, os.path.join(out_dir, 'out_numpy_array.udb'), x_col='X', y_col='Y')
    if result is not None:
        if isinstance(result, str):
            if isinstance(result, str):
                print('从 numpy 数组中写入数据到矢量数据集 %s 成功' % result)
            else:
                print('从 numpy 数组中写入数据到矢量数据集 %s 成功' % result.name)
    else:
        print('从 numpy 数组中写入数据到矢量数据集失败')


def raster_to_numpy_test():
    """从栅格数据 DEM 中读取数据到 numpy 数组中"""
    narray = datasetraster_to_numpy_array(os.path.join(example_data_dir, 'example_data.udb/DEM'))

    if narray is None:
        print('读取栅格数据集到 numpy 数组失败')
    else:
        print('ndarray.ndim : ' + str(narray.ndim))
        print('ndarray.dtype : ' + str(narray.dtype))
        print(narray)
        print('读取栅格数据集到 numpy 数组成功')

    numpy_array_to_datasetraster(narray, 0.001, 0.001, os.path.join(out_dir, 'out_numpy_array.udb'), as_grid=False)


def numpy_to_raster_test():
    """从 numpy 的数组二进制文件中加载数据,写道栅格数据集中"""

    narray = np.load(os.path.join(example_data_dir, 'dem.npy'))
    print(narray)

    result = numpy_array_to_datasetraster(narray, 0.001, 0.001, os.path.join(example_data_dir, 'out_numpy_array.udb'),
                                          as_grid=True)

    if result is not None:
        if isinstance(result, str):
            print('从 numpy 数组中写入数据到栅格数据集 %s 成功' % result)
        else:
            print('从 numpy 数组中写入数据到栅格数据集 %s 成功' % result.name)
    else:
        print('从 numpy 数组中写入数据到栅格数据集失败')


if __name__ == '__main__':
    # 读取矢量数据集到 numpy 数组成功
    vector_to_numpy_test()

    # 从 numpy 数组中写入数据到矢量数据集中成功
    numpy_to_vector_test()

    # 读取栅格数据集到 numpy 数组成功
    raster_to_numpy_test()

    # 从 numpy 数组中写入数据到栅格数据集中成功
    numpy_to_raster_test()


In [ ]:
exit()